In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_csv('cleaned_aqi_data.csv')
print(df.head())

   state_code  county_code  year  parameter_code  first_max_value  \
0           6            1  2018           42101              3.3   
1           6            1  2018           42101              3.6   
2           6            1  2018           42101              2.1   
3           6            1  2018           42101              2.3   
4           6            1  2018           42101              2.6   

   arithmetic_mean  observation_count  primary_exceedance_count  \
0         0.414305               8354                       0.0   
1         0.515045               8315                       0.0   
2         0.454929               8287                       0.0   
3         0.528526               6289                       0.0   
4         0.566017               8004                       0.0   

   ninetieth_percentile County_ID Pollutant  Pct_Unhealthy  Max_Last_Year  \
0                   0.7    06_001        CO            0.0            NaN   
1                   0.9    0

In [ ]:
clustering_data = df.groupby('County_ID').agg({
    'arithmetic_mean': 'mean',         
    'ninetieth_percentile': 'mean',    
    'primary_exceedance_count': 'mean', 
    'first_max_value': 'max'            
}).fillna(0)

clustering_data.columns = ['pollution_avg', 'bad_day_level', 'very_bad_days_count', 'worst_ever_surge']

In [10]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(clustering_data)

In [11]:
kmeans = KMeans(n_clusters=3, random_state=42)
clustering_data['Cluster'] = kmeans.fit_predict(data_scaled)

In [12]:
sil_score = silhouette_score(data_scaled, clustering_data['Cluster'])
print(f"Cluster Quality: {sil_score:.3f}")

cluster_means = clustering_data.groupby('Cluster')['very_bad_days_count'].mean()
high_risk_cluster_id = cluster_means.idxmax()

clustering_data['actually_dangerous'] = clustering_data['very_bad_days_count'] > 5 
clustering_data['model_said_dangerous'] = clustering_data['Cluster'] == high_risk_cluster_id

correct_predictions = (clustering_data['actually_dangerous'] == clustering_data['model_said_dangerous']).sum()
total_predictions = len(clustering_data)
accuracy = correct_predictions / total_predictions

print(f"Model Accuracy: {accuracy:.2%}")
print(f"Model Inertia: {kmeans.inertia_:.2f}")

Cluster Quality: 0.551
Model Accuracy: 79.82%
Model Inertia: 363.72
